<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-T/blob/master/DataAlgo_T(011)_KnightTour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Problem Solving 

From this section, we leave graph and start learning problem-solving.
It depends on the given problem, so just pick up famous problems and see how hard to obtain the solution(s) to the given problem even when we can invent algorithm for that. The reason to make the solution hard is --- computation.

# 6.1. Bruteforce Search

Brute-force searh is also known as  exhausive search.  
Sometimes the algorithm to find the solution might be hard to invent. 
In such a case, we may prpare rather simple algorithm. It can be easier to prepare, but in return,  it can produce solution candidates. 

To see which is really the solution, we need to run the verification process that fits the problem definition.

Brute-force search can be taken as a solution finding approach when the those conditions are satisfined.

* The algorithm is rather simple and it is possible to implement
* It can produce solution candidates that surely includes the solution
* Verification process is not time-consuming
* The number of solution candidates is within computational amount

Brute-force approach is also sometimes used to evaluate the difficulty of the problem-solving. If the number of the solution candidates is rapidly growing (combinatorial explosion),  the associated problem is thought to be difficult.

>Brute-force search  
https://en.wikipedia.org/wiki/Brute-force_search 

# 6.2. Backtrack Search

A backtracking approach is also a algorithm to find the solution to the given problem. Unlike the brute-force approach, generation of solution candidates will be stopped once it is obvious that the candidates cannot meet the problem requirements and going back to find other candidates.

To help your undestanding these two, we visit some very famouse examples from now.

>Backtracking  
https://en.wikipedia.org/wiki/Backtracking

# 6.2.1. Knight's Tour

Learn knight's tour problem and understand that there are problems for which we can develop the algorithm and we know it can surely search the answer, but yet it is practically impossible to get the answer due to the computation limit.
 

**Reminder**  
On github, rendering might not be in good shape.  
To see the expected layout, open this page in Google Colaboratory.
To run one specific code cell in colab, click the icon on the left part or just type Ctrl + Enter.  

# Preparation

Connect the Jupyter environment and invoke a runtime. 
Mount your Google Drive by the procedure below.  
Change directory to the mounted point and make it as the working folder.  
By then, files are preserved even after you terminate the runtime environment.

In [ ]:
!echo "Mounting your Google Drive"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "Make a working folder and chnage directory to it"
%cd /content/drive/My\ Drive
%mkdir -p DataAlgo-T/011
%cd       DataAlgo-T/011
!ls

# Knight's tour

**Problem definition**

Knight of chess game can move to one of the eight positions. There are at 2 cells moving straight to back/forth/left/right then turn squarely and move 1 more cell.

The knight's tour problem can be defined to investigate it is possible
1. for a given board (normaly 8 by 8 )
2. by following the moves allowed to knight
3. by visiting each cell only once
4. by moving ceratin times (63 times for 8x8 board size)
5. To visit all the cells

if the answer is yes, the number of the answers could be another quesiton.

If the knight can come back to the beginning cell for the next move after the tour is finished, it is called knight's tour round, or closed knight's tour.

For the knight's tour round problem, the starting position does not make sense.


# Brute-force search for knight's tour

To generate the solution candidates, think the conditions of No.2 and No.4 only for organizing a brute-force search.

Assum that the the knight is placed in a given position. By following the condition No.2 and No.4, a seris of 63 moves wher each move is in a choice of 8 positions can be easily generated. 
Then, as for verification process, the candidate is examined surely covers all the cells of the board (condition No.1, No.3, and No.5).

Apparently, this approach produces enormus number of solution candidates. As for 8x8 board case, the number of the solution candidates should be 8<sup>63</sup>.

> 8<sup>63</sup> = 2<sup>189</sup> >= 10<sup>189 x 0.3010</sup> = 10<sup>56.889</sup>

Even when we have a super computer that can verify the candidate by one instruction and it runs 1.0 THz with one billion processing nodes, 

> (10<sup>12</sup> x 60 x 60 x 24 x 365.2422) x 10<sup>9</sup> <= 3.15 x 10<sup>28</sup>

candidats can be verified per year. It means we need 10<sup>57</sup> / (3.15 x 10<sup>28</sup>) years... more than 10<sup>28</sup> years. Remember, our universe is just about 1.38 x 10<sup>10</sup> year-old. 






# C program of the blute-force search

**Purpose**

Specify the board width (boardsize.x) and height (boardsize.y) and the position (x,y) of the starting cell as 4 command line options. The program should generate solution candidates and count the number of the answers of knight's tour (and knight's tour round).

**Explanation**

vec2i structure is used to express 2D coord of the board.
Generate (boardsize.x x boardsize.y) - 1 moves as a candidate and verify the conditions for each candidate.
We adopt depth-first search to generate the candidates.

**Program**

kpath[] array stores all the moves.  
Unfortunately, in most of the cases, the path goes beyond the board.  
So only the moves within the board are visualized in board_from_path() function.
The initial position is denoted by 1.

**Remarks**

To help monitoring the process behavior, you can control the detailness of output by changing verboselevel variable. It can be specified by the 5th command line option, from 0 (default) to 3 (detailed max). Be carefull the total amount of output text comes to ENORMUS if you specify not-a-small board size (size beyond 4 x 3 is really dangeos, almost deadly!).

In [ ]:
%%writefile knighttour-bluteforce_E.c
// Knight tour by bluteforce search
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>  // printf
#include <stdlib.h> // atoi

#define M 8 // choices of one move of knight

// 2D coord.
typedef struct {int x; int y;} vec2i;

// choices of one move of knight (relative position)
vec2i move[M] = {
	{ 2 ,1},
	{ 1, 2},
	{-1, 2},
	{-2, 1},
	{-2,-1},
	{-1,-2},
	{ 1,-2},
	{ 2,-1}
};

// path memory
int *kpath = NULL; // store the direction of each move

// Memory expression of the board
int *board = NULL; // value of position (x,y) is stored at board[y * boardsize.x + x]
vec2i boardsize = {0, 0};

// Starting position of the knight
vec2i spt = {0, 0};

int knighttour      = 0; // Number of the anwers of the knight's tour
int knighttourround = 0; // Number of the anwers of the knight's tour round
int verboselevel    = 0; // Verbose level (0/min/default to 4/max)

// Memory access to the board position
int boardindex(vec2i pt){
	return (pt.y * boardsize.x + pt.x);
}

// Render the board and estimate the knight position at s-th move from path info
//    At the s-th move
//    First move (position) is indicated by spt, so remaining moves are s-1
//    This means the path info of kpath[0], ... , kpath[s-2] should be refered.
vec2i board_from_path(int s){
    int c;
	vec2i pt;
 
    for (c = 0; c < boardsize.x * boardsize.y; c++) {
        board[c] = 0; // No access
    }
    board[boardindex(spt)] = 1; // First move
    pt = spt;
    if (verboselevel >= 3) printf("** - (%d, %d) - at %d\n", pt.x, pt.y, s);
    for (c = 0; c < s - 1; c++) {
        pt.x += move[kpath[c]].x;
        pt.y += move[kpath[c]].y;
        if (verboselevel >= 3) printf(">> %d (%d, %d) %d at %d\n", c, pt.x, pt.y, kpath[c], s);
        if (pt.x >= 0 && pt.x < boardsize.x &&
            pt.y >= 0 && pt.y < boardsize.y ) {
            board[boardindex(pt)] = c + 2; // Render the move number only when it is within the board
        }
    }
    return pt;
}
 
// Display the board (x horizontal, y vertical; positive is downward)
void showboard(void){
    vec2i pt;
 	for (pt.y = 0; pt.y < boardsize.y; pt.y++) {
		for (pt.x = 0; pt.x < boardsize.x; pt.x++) {
			printf("%2d ", board[boardindex(pt)]);
		}
		printf("\n");
	}
	return ;
}

// Detection of Knight's tour
int check_knighttour(void){
	int c;
    
    for (c = 0; c < boardsize.x * boardsize.y; c++) {
        if (board[c] <= 0)
            return 0;
    }
	return 1;
}

// Detection of Knight's tour round
int check_knighttourround(vec2i pt){
	int d;
	for (d = 0; d < M; d++) {
		if (pt.x + move[d].x == spt.x && 
		    pt.y + move[d].y == spt.y) {
			return 1;
		}
	}
	return 0;
}

// Generate the solution candidates by brute-foce approach by recursive call
//   int n_move; number of moves to examine here
void knighttouronestep_bf(int n_move){
    vec2i cpt;
	int d; // choice of direction

    if (verboselevel >= 2) printf("-- %d cells fixed --\n", n_move);

    cpt = board_from_path(n_move);
    if (verboselevel >= 2) showboard();

	// Solution verification
	if (n_move == boardsize.x * boardsize.y) {
        // Solution of knight't tour?
        if (check_knighttour() > 0) {
            knighttour++;
            // Solution of knight't tour round?
            knighttourround += check_knighttourround(cpt);
            // Display the solution
		    if (verboselevel >= 1) {
			    printf("Answer found (%d / %d).\n", knighttour, knighttourround);
	    		showboard();
	    	}
        }
        return ; // Do not exit, keep working to find next solutions
	}
	
	// Search by recursive call
	for (d = 0; d < M; d++) {
		kpath[n_move - 1] = d; // Memory this move
		knighttouronestep_bf(n_move + 1); // recursive call ; depth first
	}
	return ;
}

// Main function
//   Augment: x-size of the board, y-size of the board, starting x-position, starting y-position, verbose level(0-3)
int main(int argc, char *argv[]){

	// Check command line options
    if (argc != 6) {
        printf("Speciy x-size of the board, y-size of the board, starting x-position, starting y-position, and verbose level(0-3).\n");
        printf("Quiting because of invalid options.\n");
        return -1;
    }
    // board size
	boardsize.x = atoi(argv[1]);
	boardsize.y = atoi(argv[2]);
    // starting position
	spt.x = atoi(argv[3]);
	spt.y = atoi(argv[4]);
	// verbose level
	verboselevel = atoi(argv[5]);
	printf("board_size = (%d, %d), start_point = (%d, %d), verbose level = %d\n", boardsize.x, boardsize.y, spt.x, spt.y, verboselevel);
	if (boardsize.x < 0 || boardsize.y < 0 || 
	    spt.x < 0 || spt.x >= boardsize.x || 
		spt.y < 0 || spt.y >= boardsize.y) {
		printf("Invalid value(s) on options. Force quiting.\n");
	}
	if ((kpath = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("Memory allocation failure for %d elements for the path memory. Exiting.\n", boardsize.x * boardsize.y);
		return -2;
	}
	if ((board = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("Memory allocation failure for %d elements for the board memory. Exiting.\n", boardsize.x * boardsize.y);
		return -2;
	}
	knighttouronestep_bf(1); // Start the search!

	printf("Result:\n");
	printf("  Knight tour       %6d\n", knighttour);
	printf("  Knight tour round %6d\n", knighttourround);

	return 0;
}


Compile it and check no errors.  
(You should think of optimization here because the speed does matter here.)


In [ ]:
!gcc -Wall -o knighttour-bluteforce_E knighttour-bluteforce_E.c

To measure the execution time, use the time command on linux.  
Just add "time" ahead of normal command run.  
In the result, user property corresponds to the time the process actually use the CPU.

**Caution**

While you are checking the behavior of the program, board size should be minimum 3x3.  
Of course, no answer is found on 3x3 board.  
Probably it runs for 1 second or so (as of 2020/06).  

On brute-force approach, anyhing explodes VERY SOON with the size of the board.

For example, you should estimate well about computation (time, output text amount, etc...) before you go to 4x3 and more.

In [ ]:
!time ./knighttour-bluteforce_E 3 3 0 0 1

If the verbose level is set to high, your browser get stacked pretty soon (google colaboratory with google chrome may handle hundreds lines per output, but no more for safe operations). 
To see the output with large size, redirect the standard output to a file on a google drive, and see the file by downloading it to your computer after the program ends.  
But be careful, this approach is still DANGEROUS.  
For example, at boardsize 3x3 with verbose level 2,the total output text comes to 914MB! (bluteforce33-00-2.txt below).


In [ ]:
!time ./knighttour-bluteforce_E 3 3 0 0 1

Verbose level 2 probably takes 45 sec or so (much longer than verbose level 1).

In [ ]:
!time ./knighttour-bluteforce_E 3 3 0 0 2 > bluteforce33-00-2.txt

Check the size by ls -sh and count the number of the lines by wc -l.

In [ ]:
!ls -sh bluteforce33-00-2.txt
!wc -l bluteforce33-00-2.txt

If you want, you can see only first lines by head command (100 lines in the example below).  
(If you output whole the text, surely your browser is stacked.)

In [ ]:
!head -100 bluteforce33-00-2.txt

Below is the example of running with board size 4x3.  
BEFORE YOU RUN, MAKE SURE THE ESTIMATION TIME HOW LONG YOU NEED!  
(if you spent 1.5 seconds for 3-3-0-0-1 case, what will happen on 4-3-0-0-1 case? Hint: The board size is 3x3=9 vs 4x3=12, and every move causes 8 possible moves to examine.)

In [ ]:
!time ./knighttour-bluteforce_E 4 3 0 0 1

Say, the estimation should be 1.5 x 8<sup>3</sup> seconds. How much was the result?  
OK, next example is the 4x4 board case.  
MAKE SURE HOW LONG IT WILL TAKE TO THE ENDS.  
ARE YOU REALLY OK TO RUN IT?

In [ ]:
!time ./knighttour-bluteforce_E 4 4 0 0 1 > bluteforce44-00-1.txt

Once again, 1.5 x 8<sup>(4x4-3x3)</sup> = 1.5 x 8<sup>7</sup> sec...

# Backtracking search for knight's tour

On brute-force approach, apparently there are useless candidates on generating the candidates by depth-first (recursive) search.  
Backtracking approach gives up generating the candidates if the conditions cannot be met and going back (backtrack) to find other candidates. 

On the knight's tour, the program should backtracks if one of two cases is found:

* Knight goes out of the board
* Knight comes to the cell where it has already visited

Note that it surely eliminates the useless candidates and contributes the speed up. But it usually won't change the computation amount on big-O notation viewpoint (because you have to think of the worst case on the big-O notation).


# C program of the backtracking search

**Purpose**

Speed up of searching the solutions of knight' tour by backtrack method.

**Explanation**

The program goes deeper (on DFS recursive call) only when it does not face the two cases shown above.  
In other words, it goes deeper only when the next move is within the board, and it is a unvisited cell.

On writing thr backtracking search, the important issue is to **write back** the variables to the original state when it backtracks.

**Program**

Line 96 and 98 should be pairwised on writing the backtracking program.
Line 96 is on the go, and Line 98 is on the way back (backtracking).

**Remarks**

The verboselevel variable now accepts from 0 to 2.


In [ ]:
%%writefile knighttour-backtrack_E.c
// Knight tour by backtrack search
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>  // printf
#include <stdlib.h> // atoi

#define M 8 // choices of one move of knight

// 2D coord.
typedef struct {int x; int y;} vec2i;

// choices of one move of knight (relative position)
vec2i move[M] = {
	{ 2 ,1},
	{ 1, 2},
	{-1, 2},
	{-2, 1},
	{-2,-1},
	{-1,-2},
	{ 1,-2},
	{ 2,-1}
};

// Memory expression of the board
int *board = NULL; // value of position (x,y) is stored at board[y * boardsize.x + x]
vec2i boardsize = {0, 0};

// Starting position of the knight
vec2i spt = {0, 0};

int knighttour      = 0; // Number of the anwers of the knight's tour
int knighttourround = 0; // Number of the anwers of the knight's tour round
int verboselevel    = 0; // Verbose level (0/min/default to 2/max)

// Memory access to the board position
int boardindex(vec2i pt){
	return (pt.y * boardsize.x + pt.x);
}

// Display the board (x horizontal, y vertical; positive is downward)
void showboard(void){
    vec2i pt;
 	for (pt.y = 0; pt.y < boardsize.y; pt.y++) {
		for (pt.x = 0; pt.x < boardsize.x; pt.x++) {
			printf("%2d ", board[boardindex(pt)]);
		}
		printf("\n");
	}
	return ;
}

// Detection of Knight's tour round
int check_knighttourround(vec2i pt){
	int d;
	for (d = 0; d < M; d++) {
		if (pt.x + move[d].x == spt.x && 
		    pt.y + move[d].y == spt.y) {
			return 1;
		}
	}
	return 0;
}

// Generate the solution candidates by backtrack approach by recursive call
//   vec2i cpt; current position
//   int n_move; number of fixed moves
void knighttouronestep_bt(vec2i cpt, int n_move){
	vec2i npt; // next point
	int d; // choice of direction

    if (verboselevel >= 2) printf("-- %d cells fixed --\n", n_move);
	if (verboselevel >= 2) showboard();

	// Solution verification
	if (n_move == boardsize.x * boardsize.y) { 
	    // Solution of knight't tour
		knighttour++; 
		// Solution of knight't tour round?
		knighttourround += check_knighttourround(cpt);
		// Display the solution
		if (verboselevel >= 1) {
			printf("Answer found (%d / %d).\n", knighttour, knighttourround);
			showboard();
		}
		return ; // Do not exit, keep working to find next solutions
	}
	
	// Search by recursive call
	for (d = 0; d < M; d++) {
		// Next position
		npt.x = cpt.x + move[d].x;
		npt.y = cpt.y + move[d].y;
		if (npt.x >= 0 && npt.x < boardsize.x &&
		    npt.y >= 0 && npt.y < boardsize.y &&
			board[boardindex(npt)] == 0) {
			board[boardindex(npt)] = n_move + 1;   // Memory this move (original value is 0)
			knighttouronestep_bt(npt, n_move + 1); // recursive call ; depth first
			board[boardindex(npt)] = 0;            // WRITE BACK to the original state on BACKTRACKING
		}
	}
	return ;
}

// Main function
//   Augment: x-size of the board, y-size of the board, starting x-position, starting y-position, verbose level(0-2)
int main(int argc, char *argv[]){

	// Check command line options
    if (argc != 6) {
        printf("Speciy x-size of the board, y-size of the board, starting x-position, starting y-position, and verbose level(0-2).\n");
        printf("Quiting because of invalid options.\n");
        return -1;
    }
    // board size
	boardsize.x = atoi(argv[1]);
	boardsize.y = atoi(argv[2]);
    // starting position
	spt.x = atoi(argv[3]);
	spt.y = atoi(argv[4]);
	// verbose level
	verboselevel = atoi(argv[5]);
	printf("board_size = (%d, %d), start_point = (%d, %d), verbose level = %d\n", boardsize.x, boardsize.y, spt.x, spt.y, verboselevel);
	if (boardsize.x < 0 || boardsize.y < 0 || 
	    spt.x < 0 || spt.x >= boardsize.x || 
		spt.y < 0 || spt.y >= boardsize.y) {
		printf("Invalid value(s) on options. Force quiting.\n");
	}
	if ((board = calloc(boardsize.x * boardsize.y, sizeof(int))) == NULL) {
		printf("Memory allocation failure for %d elements for the board memory. Exiting.\n", boardsize.x * boardsize.y);
		return -2;
	}
	board[boardindex(spt)] = 1; // Starting position
	knighttouronestep_bt(spt, 1); // Start the search!

	printf("Result:\n");
	printf("  Knight tour       %6d\n", knighttour);
	printf("  Knight tour round %6d\n", knighttourround);

	return 0;
}


Compile it and check no errors.  
It is also a very good idea to comple it with high optimization.

In [ ]:
!gcc -Wall -o knighttour-backtrack_E knighttour-backtrack_E.c

Run it.  
See how much faster than the brute-force program.  
You can try larger board size with this backtrack program.

In [ ]:
!time ./knighttour-backtrack_E 6 6 0 0 0

# Handling large values

When you write programs that may include combinatorial explosion, you have to be very careful on many issues.  
One typical issue is the upper limit of the variables.  
On the above progam, the type of the counter of the answers is integer, but it might not be good.  
The number of the candidates is 8<sup>63</sup> for board 8x8, so it might be a chance of the number of the answers come close to 8<sup>63</sup>  (who knows?).  
Since 8<sup>63</sup> = 2<sup>189</sup>, it needs 189 bits whie the unsigned long int corresponds to only 64 bits.  
So you need to prepare special tricks just to count numbers in combinatorial explosion situation...

(If you know the number shall be  below 64 bits, it is OK, but how you know it in actual cases??)

In [ ]:
%%writefile biginteger.c
#include <stdio.h>
#include <limits.h>
int main(int argc, char *argv[]){
    printf("type int           : Max value is INT_MAX   %d\n", INT_MAX);
    printf("type long          : Max value is LONG_MAX  %ld\n", LONG_MAX);
    printf("type unsigned long : Max value is ULONG_MAX %lu\n", ULONG_MAX);
    printf("type unsigned long : number of the bits used is %ld\n", sizeof(unsigned long) * 8);
    return 0;
}


In [ ]:
!gcc -o biginteger biginteger.c
!./biginteger

# Problems

1. Computation amount on brute-force  
Discuss the computation amount of time and space of knighttour-bluteforce_E.c program. Take the number of the cells of the board as input N.

2. Computation amount on backtrack  
Discuss the computation amount of time and space of knighttour-backtrack_E.c program. Take the number of the cells of the board as input N.

3. Verbose level
Explain the verbose level in knighttour-bluteforce_E.c program and knighttour-backtrack_E.c program. What you can see at each level?

4. Computation time investigation  
Find out how to opitimize on gcc compiling. Optimize knighttour-bluteforce_E.c program and knighttour-backtrack_E.c program. Then take up the situation where it needs more than one minutes to the end in the origina binary, and check the speed up extent by the optimization.




#**Course Info**

https://github.com/kameda-yoshinari/DataAlgo-T  
Course: Data structure and algorithm  
Department of Engineering Systems, University of Tsukuba,Japan.  
Author: KAMEDA, Yoshinari  
2020.05.19. -